In [1]:
import numpy as np
import os, sys
import matplotlib.pyplot as plt

%matplotlib ipympl

In [2]:
# Extract data from Numpy files
chem_avg_savename = 'extracted_chemistry_avg.npy'
chem_stddev_savename = 'extracted_chemistry_stddev.npy'
chem_names_savename = 'extracted_chemistry_oxide_names.npy'
chem_spec_names_savename = 'extracted_chemistry_spec_names.npy'
chem_spec_ids_savename = 'extracted_chemistry_spec_ids.npy'


avgs = np.load(chem_avg_savename, allow_pickle=True)
stddevs = np.load(chem_stddev_savename, allow_pickle=True)
names_oxides = np.load(chem_names_savename, allow_pickle=True)
names = np.load(chem_spec_names_savename, allow_pickle=True)
ids = np.load(chem_spec_ids_savename, allow_pickle=True)


print("averages shape: ", avgs.shape)
print("stddevs shape: ", stddevs.shape)
print("oxide names shape: ", names_oxides.shape)
print("spectrum names shape: ", names.shape)
print("spectrum ids shape: ", ids.shape)

averages shape:  (1315, 20)
stddevs shape:  (1315, 20)
oxide names shape:  (20,)
spectrum names shape:  (1315,)
spectrum ids shape:  (1315,)


In [9]:
# Look at the sums of the oxides to see if any greater than 100 or close to 0
total_wt_percent = np.sum(avgs, axis=1)
idxs_gt_100 = np.where(total_wt_percent > 101)
spec_wt_percent_gt_100 = total_wt_percent[idxs_gt_100]
spec_names_gt_100 = names[idxs_gt_100]
spec_ids_gt_100 = ids[idxs_gt_100]
spec_avgs_gt_100 = avgs[idxs_gt_100]

#print("\n{}\t{}\t{}".format("weight %", "sample id", "sample name"))
name_str = ", ".join("{}".format(n) for n in names_oxides)
#print(name_str)

for i in range(len(spec_names_gt_100)):
    spec_name = spec_names_gt_100[i]
    spec_id = spec_ids_gt_100[i]
    spec_wt_percent = spec_wt_percent_gt_100[i]
    spec_avg = spec_avgs_gt_100[i]
    
    print("\n{}\t{}\t{}".format("weight %", "sample id", "sample name"))
    print("{:.4f}\t{}  \t{}".format(spec_wt_percent, spec_id, spec_name))
    
    for j in range(len(names_oxides)):
        print("{}\t{:.4f}".format(names_oxides[j], spec_avg[j]))

    
    #spec_avg_text = ", ".join(f"{x:.2f}" for x in spec_avg)
    #print("{:.4f}\t{} \t{}\n{}".format(spec_wt_percent, spec_id, spec_name, spec_avg_text))
    #print(spec_avg_text)



weight %	sample id	sample name
101.2000	X050014  	Albite
SiO2	59.2000
TiO2	0.0000
Al2O3	26.6000
MgO	0.0000
MnO	0.0000
CaO	7.9800
Na2O	7.3700
K2O	0.0500
P2O5	0.0000
ZnO	0.0000
SnO	0.0000
H2O	0.0000
Cr2O3	0.0000
SO3	0.0000
TeO2	0.0000
PbO	0.0000
CuO	0.0000
FeOT	0.0000
Fe2O3	0.0000
FeO	0.0000

weight %	sample id	sample name
113.1997	R070358-2  	Barbosalite
SiO2	0.0000
TiO2	0.0000
Al2O3	0.0480
MgO	0.0120
MnO	0.6160
CaO	0.0110
Na2O	0.0000
K2O	0.0000
P2O5	36.1030
ZnO	0.0000
SnO	0.0000
H2O	0.0000
Cr2O3	0.0000
SO3	0.0100
TeO2	0.0000
PbO	0.0000
CuO	0.0000
FeOT	0.0000
Fe2O3	58.7690
FeO	17.6307

weight %	sample id	sample name
102.7060	R060287-2  	Diaspore
SiO2	0.0173
TiO2	0.0620
Al2O3	84.2047
MgO	0.0020
MnO	0.0060
CaO	0.0000
Na2O	0.0000
K2O	0.0000
P2O5	0.0000
ZnO	0.0000
SnO	0.0000
H2O	17.8880
Cr2O3	0.0000
SO3	0.0000
TeO2	0.0000
PbO	0.0000
CuO	0.0000
FeOT	0.0000
Fe2O3	0.0000
FeO	0.5260

weight %	sample id	sample name
102.2479	R060546-2  	Diaspore
SiO2	0.0250
TiO2	0.0636
Al2O3	84.3914
MgO	0.0029
M

In [58]:
# Check iron to see if both FeO and Fe2O3 are high b/c some excel files recalculate oxide % for FeO and Fe2O3 
# We will combine and use FeOT only
feot = avgs[:,-3]
fe2o3 = avgs[:,-2]
feo = avgs[:,-1]

idxs_feot = np.where(feot>0)[0]
print(idxs_feot)
print("No FeOT values found")

idxs_fe = np.where((feot > 0) | ((fe2o3 > 0.1) & (feo > 0.1)))[0]

spec_wt_percent_fe = total_wt_percent[idxs_fe]
spec_names_fe = names[idxs_fe]
spec_ids_fe = ids[idxs_fe]

name_str = "\t".join("{}".format(n) for n in names_oxides)

print("\n{}\t{}\t{}\t\t{}\t{}\t{}".format("weight %", "sample id", "sample name", "FeOT", "Fe2O3", "FeO"))


for i in idxs_fe:
    spec_name = names[i]
    spec_id = ids[i]
    spec_wt_percent = total_wt_percent[i]
    spec_feot = feot[i]
    spec_fe2o3 = fe2o3[i]
    spec_feo = feo[i]

    print("{:.2f}\t\t{}\t{}\t\t{:.3f}\t{:.2f}\t{:.2f}".format(spec_wt_percent, spec_id, spec_name[0:15], spec_feot, spec_fe2o3, spec_feo))

for i in idxs_fe:
    spec_name = names[i]
    spec_id = ids[i]
    spec_wt_percent = total_wt_percent[i]
    spec_feot = feot[i]
    spec_fe2o3 = fe2o3[i]
    spec_feo = feo[i]

    spec_avg_text = "\t".join(f"{x:.2f}" for x in avgs[i])

    #print("{:.2f}\t\t{}\t{}".format(spec_wt_percent, spec_id, spec_name[0:15]))

    #print(name_str)
    #print(spec_avg_text)

[]
No FeOT values found

weight %	sample id	sample name		FeOT	Fe2O3	FeO
113.20		R070358-2	Barbosalite		0.000	58.77	17.63
97.39		R050109-2	Bobdownsite		0.000	0.93	0.28
100.08		R061026-2	Cronstedtite		0.000	35.29	38.23
86.98		R100208-2	Despujolsite		0.000	1.03	0.93
99.04		R040019-2	Fluororichterit		0.000	1.29	2.31
88.35		R060580-2	Jinshajiangite		0.000	1.50	26.18
105.55		R040010-2	Magnesiohasting		0.000	6.76	15.47
85.03		R070599-2	Natrodufrenite		0.000	42.73	7.77
100.52		R060858-2	Rockbridgeite		0.000	49.52	6.72
nan		R070553-2	Sarcopside		0.000	42.85	45.46
nan		R060954-2	Ushkovite		0.000	38.23	34.40
121.50		R140111-2	Kryzhanovskite		0.000	36.98	33.27
100.77		R140743-2	Manganbabington		0.000	16.00	2.04
98.04		R141210-2	Marsturite		0.000	0.16	0.15
118.61		R160033-2	Varulite		0.000	19.60	17.64
